In [ ]:
!pip install numpy
!pip install numba

In [ ]:
%%time
import numpy as np
from numba import cuda

# Problem parameters
a = 110.0
length = 2000.0
time = 160.0
nodes = 1600

dx = length / (nodes - 1)
dy = length / (nodes - 1)
dt = min(dx**2 / (4 * a), dy**2 / (4 * a))

t_nodes = int(time / dt) + 1

# Initialize data on the host
u = np.zeros((nodes, nodes), dtype=np.float64) + 20.0
u[0, :] = np.linspace(0, 100, nodes, dtype=np.float64)
u[-1, :] = np.linspace(0, 100, nodes, dtype=np.float64)
u[:, 0] = np.linspace(0, 100, nodes, dtype=np.float64)
u[:, -1] = np.linspace(0, 100, nodes, dtype=np.float64)

# Move data to device
u_d = cuda.to_device(u)
w_d = cuda.device_array_like(u_d)

# Define CUDA kernels
@cuda.jit
def copy_kernel(u, w, nodes):
    i, j = cuda.grid(2)
    if i < nodes and j < nodes:
        w[i, j] = u[i, j]

@cuda.jit
def timestep_kernel(u, w, dx, dy, dt, a, nodes):
    i, j = cuda.grid(2)
    if i > 0 and i < nodes - 1 and j > 0 and j < nodes - 1:
        dd_ux = (w[i - 1, j] - 2.0 * w[i, j] + w[i + 1, j]) / (dx * dx)
        dd_uy = (w[i, j - 1] - 2.0 * w[i, j] + w[i, j + 1]) / (dy * dy)
        u[i, j] = dt * a * (dd_ux + dd_uy) + w[i, j]

# Configure GPU kernel dimensions
threadsperblock = (16, 16)
blockspergrid_x = (nodes + threadsperblock[0] - 1) // threadsperblock[0]
blockspergrid_y = (nodes + threadsperblock[1] - 1) // threadsperblock[1]
blockspergrid = (blockspergrid_x, blockspergrid_y)

counter = 0.0

# Time-stepping loop
while counter < time:
    # Copy u into w on the GPU
    copy_kernel[blockspergrid, threadsperblock](u_d, w_d, nodes)
    
    # Perform one timestep
    timestep_kernel[blockspergrid, threadsperblock](u_d, w_d, dx, dy, dt, a, nodes)
    
    cuda.synchronize()
    counter += dt

# Copy the result back to host
u_d.copy_to_host(u)
